In [221]:
import binascii
import numpy as np
import cv2
import os
import ffmpeg
from PIL import Image

In [222]:
def strip(content):
    curx = str(content)[2:len(content)]
    return curx

In [223]:
# def form():
#     video_name = './vid.mp4'
#     images = ['1.jpg','2.jpg']
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     video = cv2.VideoWriter(video_name, fourcc, 10, (1280,720))
#     for image in images:
#         video.write(cv2.imread(image))
#     cv2.destroyAllWindows()
#     video.release()
def form_using_image_folder(image_folder):
    video_name = './vid.avi'
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    fourcc = 0
    video = cv2.VideoWriter(video_name, fourcc, 10, (1920,1080))
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video.release()

In [224]:
image_folder = './folder_img' 
form_using_image_folder(image_folder)

In [225]:
(
    ffmpeg.input("vid.avi")
            .output("output.avi",vcodec = 'h264')
            .run()
)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

(None, None)

In [226]:
vid = cv2.VideoCapture('./output.avi')
success,image = vid.read()
c = 0
while success:
  cv2.imwrite("./out_check/Frame%05d.jpg" % c, image)    
  success,image = vid.read()
  print('Reading frame: ', c)
  c = c + 1

print('done')


Reading frame:  0
Reading frame:  1
done


![](./possible_leads%20/9.png)

We are back to 120kB from the original 250kB images, however this time there is something different.

Somehow the frames have more information than we started with.

In [227]:
filename = 'output.avi'
with open(filename, 'rb') as f:
    content = f.read()
content=binascii.hexlify(content)
# print(content)

In [234]:
input_images_1 = Image.open('./out_check/Frame00000.jpg')
input_images_2 = Image.open('./folder_img/2.jpg')
pixel_map_1 = input_images_1.load()
pixel_map_2 = input_images_2.load()
point = (940,553)
a = pixel_map_1[point]
b = pixel_map_2[point]
consider_same = a == b
print(consider_same)
print(a)
print(b)


False
(89, 99, 98)
(92, 100, 102)


In [229]:
number_pixels = 1920*1080
threshold = 3
count = 0
for i in range (0,1920):
    for j in range (0,1080):
        point = (i,j)
        a = pixel_map_1[point]
        b = pixel_map_2[point]
        if ((a[0] in range (b[0]-threshold,b[0]+threshold)) & (a[1] in range (b[1]-threshold,b[1]+threshold)) & (a[2] in range (b[2]-threshold,b[2]+threshold))):
            count = count + 1
per_equal = (count/number_pixels)*100
print(str(per_equal)+"'%' under threshold")

52.492332175925924'%' under threshold


In [230]:
number_pixels = 1920*1080
threshold = 5
count = 0
for i in range (0,1920):
    for j in range (0,1080):
        point = (i,j)
        a = pixel_map_1[point]
        b = pixel_map_2[point]
        if ((a[0] in range (b[0]-threshold,b[0]+threshold)) & (a[1] in range (b[1]-threshold,b[1]+threshold)) & (a[2] in range (b[2]-threshold,b[2]+threshold))):
            count = count + 1
per_equal = (count/number_pixels)*100
print(str(per_equal)+"'%' under threshold")

88.88560956790124'%' under threshold


In [231]:
number_pixels = 1920*1080
threshold = 10
count = 0
for i in range (0,1920):
    for j in range (0,1080):
        point = (i,j)
        a = pixel_map_1[point]
        b = pixel_map_2[point]
        if ((a[0] in range (b[0]-threshold,b[0]+threshold)) & (a[1] in range (b[1]-threshold,b[1]+threshold)) & (a[2] in range (b[2]-threshold,b[2]+threshold))):
            count = count + 1
per_equal = (count/number_pixels)*100
print(str(per_equal)+"'%' under threshold")

98.91507523148148'%' under threshold
